In [1]:
import pandas as pd

df = pd.read_csv('../data/dataset/코스피_전처리전.csv')
df.head()

,Symbol,Name,결산월,회계년,주기,당기순이익률(%),매출총이익률(%),영업이익률(%),자본금영업이익률(%),자본금세전계속사업이익률(%),...,DSRI,GMI,AQI,SGI,DEPI,SGAI,LVGI,TATA,벤포드,분식기업
0,A005930,삼성전자,12,2011,Annual,8.34,32.03,9.48,"1,743.07","1,915.50",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.046685,0,0.0
1,A005930,삼성전자,12,2012,Annual,11.86,37.02,14.44,"3,236.64","3,333.10",...,0.894689,0.865279,0.978884,1.218797,0.010662,1.001059,0.941047,-0.049281,0,0.0
2,A005930,삼성전자,12,2013,Annual,13.33,39.79,16.08,"4,098.54","4,274.50",...,0.920906,0.930428,0.929372,1.137188,0.011489,1.049976,0.909246,-0.046350,1,0.0
3,A005930,삼성전자,12,2014,Annual,11.35,37.79,12.14,"2,788.27","3,105.80",...,1.096005,1.052893,1.148993,0.901673,0.012529,1.082266,0.904046,-0.051863,0,0.0
4,A005930,삼성전자,12,2015,Annual,9.50,38.46,13.16,"2,942.96","2,892.54",...,1.047373,0.982603,0.854211,0.973073,0.013063,0.986019,0.963436,-0.056356,0,0.0


In [2]:
print(df.shape, df.drop_duplicates('Name').shape)

(13356, 48) (1100, 48)


---
### 11년 데이터 제거

In [3]:
df = df[df['회계년'] != 2011]

In [4]:
print(df.shape, df.drop_duplicates('Name').shape)

(12243, 48) (1100, 48)


---
### 결산월 12월 이외의 데이터 제거

In [5]:
df = df[df['결산월'] == 12]

In [6]:
print(df.shape, df.drop_duplicates('Name').shape)

(12150, 48) (1099, 48)


---
### 상장 이전 회계연도 데이터 제거하기 (불필요한 데이터 제거)

In [7]:
# KRX에서 상장일 데이터 가져왔음! 불러오기!! (코스피)

kospi_info = pd.read_csv('../data/코스피_기본정보.csv', encoding='CP949')
kospi_info = kospi_info[['단축코드', '한글 종목약명', '상장일']]

# '상장일' 칼럼을 날짜 형식으로 변환
kospi_info['상장일'] = pd.to_datetime(kospi_info['상장일'], format='%Y/%m/%d')

# 연도만 남기기
kospi_info['상장일'] = kospi_info['상장일'].dt.year

# '상장일' 칼럼을 문자열로 변환
kospi_info['상장일'] = kospi_info['상장일'].astype(str)

# '단축코드' 칼럼의 모든 데이터에 'A'를 추가하기
kospi_info['단축코드'] = 'A' + kospi_info['단축코드'].astype(str)

# '단축코드' 칼럼명을 'Symbol'로 변경하기
kospi_info.rename(columns={'단축코드':'Symbol', 
                           '한글 종목약명':'Name', 
                           '상장일':'상장연도'}, inplace=True)

In [8]:
df = pd.merge(kospi_info, df, on='Symbol')

In [9]:
df.head()

,Symbol,Name_x,상장연도,Name_y,결산월,회계년,주기,당기순이익률(%),매출총이익률(%),영업이익률(%),...,DSRI,GMI,AQI,SGI,DEPI,SGAI,LVGI,TATA,벤포드,분식기업
0,A095570,AJ네트웍스,2015,AJ네트웍스,12,2012,Annual,2.90,33.81,13.57,...,0.778246,0.761427,4.128503,1.379072,0.004992,1.570106,0.958189,-0.031112,1,0.0
1,A095570,AJ네트웍스,2015,AJ네트웍스,12,2013,Annual,2.39,100.00,9.32,...,0.632011,0.338114,0.310695,4.129326,0.009748,4.480779,1.032847,0.117086,0,0.0
2,A095570,AJ네트웍스,2015,AJ네트웍스,12,2014,Annual,2.77,100.00,7.61,...,0.809239,1.000000,1.067126,1.335497,0.184039,1.018785,0.969114,0.109306,0,0.0
3,A095570,AJ네트웍스,2015,AJ네트웍스,12,2015,Annual,2.75,100.00,7.03,...,1.027277,1.000000,0.811522,1.043941,0.179920,1.006256,0.958033,0.104674,0,0.0
4,A095570,AJ네트웍스,2015,AJ네트웍스,12,2016,Annual,1.36,100.00,4.93,...,1.128408,1.000000,1.278754,1.187855,0.173645,1.022605,1.032904,0.119441,0,0.0


In [10]:
# '상장연도' 칼럼의 데이터 타입을 정수형으로 변환
df['상장연도'] = df['상장연도'].astype(int)

# '회계년' 칼럼이 '상장연도' 칼럼보다 작은 행을 필터링하여 삭제
df = df[df['회계년'] >= df['상장연도']]

# 위 코드는 '회계년' 칼럼이 '상장연도' 칼럼보다 크거나 같은 행만 유지하고, 나머지 행은 모두 삭제합니다. 
# 이렇게 하면 '회계년'이 '상장연도'보다 작은 행은 필터링되어 제거됩니다.

In [11]:
df.head()

,Symbol,Name_x,상장연도,Name_y,결산월,회계년,주기,당기순이익률(%),매출총이익률(%),영업이익률(%),...,DSRI,GMI,AQI,SGI,DEPI,SGAI,LVGI,TATA,벤포드,분식기업
3,A095570,AJ네트웍스,2015,AJ네트웍스,12,2015,Annual,2.75,100.0,7.03,...,1.027277,1.0,0.811522,1.043941,0.179920,1.006256,0.958033,0.104674,0,0.0
4,A095570,AJ네트웍스,2015,AJ네트웍스,12,2016,Annual,1.36,100.0,4.93,...,1.128408,1.0,1.278754,1.187855,0.173645,1.022605,1.032904,0.119441,0,0.0
5,A095570,AJ네트웍스,2015,AJ네트웍스,12,2017,Annual,2.45,100.0,2.66,...,1.963129,1.0,8.471571,0.673065,0.135205,1.023927,1.019441,0.072598,0,0.0
6,A095570,AJ네트웍스,2015,AJ네트웍스,12,2018,Annual,3.56,100.0,-2.03,...,0.533427,1.0,0.439056,1.238862,0.045374,1.048180,0.998717,0.024194,0,0.0
7,A095570,AJ네트웍스,2015,AJ네트웍스,12,2019,Annual,4.21,100.0,1.56,...,1.356556,1.0,1.797050,0.956704,0.071189,0.964759,0.997978,0.029701,0,0.0


In [12]:
df[df['Name_x'] != df['Name_y']]

,Symbol,Name_x,상장연도,Name_y,결산월,회계년,주기,당기순이익률(%),매출총이익률(%),영업이익률(%),...,DSRI,GMI,AQI,SGI,DEPI,SGAI,LVGI,TATA,벤포드,분식기업
2787,A286940,롯데이노베이트,2018,롯데정보통신,12,2018,Annual,2.88,8.74,4.80,...,0.182187,0.900059,0.733660,5.443658,0.001581,0.908020,0.765024,-0.097064,1,0.0
2788,A286940,롯데이노베이트,2018,롯데정보통신,12,2019,Annual,6.27,8.84,4.86,...,0.845340,0.989103,1.065710,1.041798,0.009476,1.009215,0.917151,-0.109545,0,0.0
2789,A286940,롯데이노베이트,2018,롯데정보통신,12,2020,Annual,3.52,8.11,4.56,...,0.898020,1.090343,0.941680,1.004597,0.011087,0.891420,0.827060,0.081568,0,0.0
2790,A286940,롯데이노베이트,2018,롯데정보통신,12,2021,Annual,3.73,8.13,4.30,...,1.149921,0.996758,1.255181,1.094701,0.008612,1.081829,1.161863,-0.063098,1,0.0
2791,A286940,롯데이노베이트,2018,롯데정보통신,12,2022,Annual,2.71,8.94,3.27,...,1.257414,0.909765,1.578503,1.126566,0.010341,1.478242,1.134770,0.022711,0,0.0
4892,A010620,HD현대미포,1983,현대미포조선,12,2012,Annual,1.98,8.12,2.13,...,1.570234,1.744509,0.953146,0.954906,0.016339,1.021046,0.993465,0.107007,0,0.0
4893,A010620,HD현대미포,1983,현대미포조선,12,2013,Annual,-6.70,0.08,-6.90,...,1.405241,96.949731,0.816375,0.902722,0.017147,1.165580,1.218349,0.037008,0,0.0
4894,A010620,HD현대미포,1983,현대미포조선,12,2014,Annual,-17.12,-13.42,-21.87,...,0.330402,-0.006244,0.628331,0.995394,0.016329,1.209591,1.183643,-0.126543,0,0.0
4895,A010620,HD현대미포,1983,현대미포조선,12,2015,Annual,0.55,8.73,1.43,...,2.139231,-1.536525,1.057934,1.172642,0.016933,0.864024,1.001311,0.135533,0,0.0
4896,A010620,HD현대미포,1983,현대미포조선,12,2016,Annual,1.15,10.11,5.54,...,0.214882,0.863960,0.909081,0.740786,0.014935,0.625080,0.932812,-0.036701,0,0.0


In [13]:
df.drop(['Name_y'], axis=1, inplace=True)
df.rename(columns={'Name_x': 'Name'}, inplace=True)

In [14]:
print(df.shape, df.drop_duplicates('Name').shape)

(7751, 49) (778, 49)


---
### 금융업 제외시키기 (불필요한 데이터 제거)
- WICS 기준 금융업 제외
- 은행 / 증권 / 다각화된금융 / 보험 / 부동산
- 키워드 제거

In [15]:
# 은행
bank = ['KB금융', 
        '신한지주', 
        '하나금융지주', 
        '우리금융지주', 
        '기업은행', 
        '카카오뱅크', 
        'BNK금융지주', 
        'JB금융지주', 
        'DGB금융지주', 
        '상상인', 
        '제주은행']

# result = result[~result['Name'].isin(bank)]

# 증권
securities = ['메리츠금융지주', 
              '한국금융지주', 
              '미래에셋증권', 
              '삼성증권', 
              'NH투자증권', 
              '키움증권', 
              '다우기술', 
              '대신증권', 
              '한화투자증권', 
              '유진투자증권', 
              '신영증권', 
              '유안타증권', 
              'SK증권', 
              '다우데이타', 
              '다올투자증권', 
              '교보증권', 
              '현대차증권', 
              'DB금융투자', 
              '케이프', 
              '한양증권', 
              '부국증권', 
              '이베스트투자증권', 
              '유화증권', 
              '상상인증권', 
              '코리아에셋투자증권']

# result = result[~result['Name'].isin(securities)]

# 다각화된금융
diversified = ['삼성카드', 
               '우리기술투자', 
               '스틱인베스트먼트', 
               '글로벌텍스프리', 
               '미래에셋벤처투자', 
               '아주IB투자', 
               '에이티넘인베스트', 
               '리드코프', 
               'DSC인베스트먼트', 
               'SV인베스트먼트', 
               'SBI인베스트먼트', 
               '대성창투', 
               '한국캐피탈', 
               "메이슨캐피탈", 
               'HB인베스트먼트', 
               '캡스톤파트너스', 
               '스톤브릿지벤처스', 
               '컴퍼니케이', 
               '나우IB', 
               '큐캐피탈', 
               'TS인베스트먼트', 
               '푸른저축은행', 
               'LB인베스트먼트', 
               '리더스 기술투자', 
               '린드먼아시아']

# result = result[~result['Name'].isin(diversified)]

insurance = ['삼성화재', 
             '삼성생명', 
             'DB손해보험', 
             '현대해상', 
             '한화생명', 
             '코리안리', 
             '한화손해보험', 
             '미래에셋생명', 
             '롯데손해보험', 
             '동양생명', 
             '인카금융서비스', 
             '에이플러스에셋', 
             '흥국화재']

# result = result[~result['Name'].isin(insurance)]

df = df[~df['Name'].isin(set(bank)|set(securities)|set(diversified)|set(insurance))]

In [16]:
df = df[~df['Name'].str.contains('은행|증권|투자|인베스트먼트|스팩|보험|화재|신탁|금융|카드|캐피탈|우리종금|대성창투')]

In [17]:
print(df.shape, df.drop_duplicates('Name').shape)

(7313, 49) (732, 49)


In [18]:
df.isnull().sum()

Symbol                 0
Name                   0
상장연도                   0
결산월                    0
회계년                    0
주기                     0
당기순이익률(%)              1
매출총이익률(%)              1
영업이익률(%)               1
자본금영업이익률(%)           45
자본금세전계속사업이익률(%)       45
*영업수익/영업비용(%)          0
ROE(세전계속사업이익)(%)      45
자본금지배주주순이익률(%)        45
매출액증가율(전년동기)(%)       47
총자본증가율(전년동기)(%)       45
유형자산증가율(전년동기)(%)      45
영업이익증가율(전년동기)(%)      45
당기순이익증가율(전년동기)(%)     45
유형자산구성비율(%)            0
무형자산구성비율(%)            0
*판관비율(%)               1
타인자본비율(%)              0
부채비율(%)                0
현금및현금성자산구성비율(%)        0
총자산회전율(회)             45
총부채회전율(회)             45
총자본회전율(회)             45
자본금회전율(회)             45
현금흐름/총부채(%)           45
현금흐름/총차입부채(%)        277
현금흐름/영업수익(%)           1
현금흐름/총자본(%)           45
유동비율(%)                0
당좌비율(%)                0
매출채권회전율               45
재고자산회전율               45
고정자산회전율               45
영업현금흐름/투자현금흐름(%)       0
DSRI                  68


In [21]:
df.reset_index(drop=True, inplace = True)

---
### 한글 값 처리

In [22]:
df.columns[6:-2]

Index(['당기순이익률(%)', '매출총이익률(%)', '영업이익률(%)', '자본금영업이익률(%)', '자본금세전계속사업이익률(%)',
       '*영업수익/영업비용(%)', 'ROE(세전계속사업이익)(%)', '자본금지배주주순이익률(%)',
       '매출액증가율(전년동기)(%)', '총자본증가율(전년동기)(%)', '유형자산증가율(전년동기)(%)',
       '영업이익증가율(전년동기)(%)', '당기순이익증가율(전년동기)(%)', '유형자산구성비율(%)', '무형자산구성비율(%)',
       '*판관비율(%)', '타인자본비율(%)', '부채비율(%)', '현금및현금성자산구성비율(%)', '총자산회전율(회)',
       '총부채회전율(회)', '총자본회전율(회)', '자본금회전율(회)', '현금흐름/총부채(%)', '현금흐름/총차입부채(%)',
       '현금흐름/영업수익(%)', '현금흐름/총자본(%)', '유동비율(%)', '당좌비율(%)', '매출채권회전율',
       '재고자산회전율', '고정자산회전율', '영업현금흐름/투자현금흐름(%)', 'DSRI', 'GMI', 'AQI', 'SGI',
       'DEPI', 'SGAI', 'LVGI', 'TATA'],
      dtype='object')

In [23]:
import re

columns_to_check = df.columns[6:-2]

# 각 열에 대해 고유한 값 추출하여 한글을 포함하는 값 확인
for column in columns_to_check:
    # 고유한 값 추출
    unique_values = df[column].astype(str).unique()
    # 한글을 포함하는 값 추출
    korean_values = [value for value in unique_values if re.search('[ㄱ-ㅎㅏ-ㅣ가-힣]', value)]
    print(f"'{column}'에 존재하는 문자 데이터: {korean_values}")

'당기순이익률(%)'에 존재하는 문자 데이터: []
'매출총이익률(%)'에 존재하는 문자 데이터: []
'영업이익률(%)'에 존재하는 문자 데이터: []
'자본금영업이익률(%)'에 존재하는 문자 데이터: []
'자본금세전계속사업이익률(%)'에 존재하는 문자 데이터: []
'*영업수익/영업비용(%)'에 존재하는 문자 데이터: []
'ROE(세전계속사업이익)(%)'에 존재하는 문자 데이터: ['완전잠식']
'자본금지배주주순이익률(%)'에 존재하는 문자 데이터: []
'매출액증가율(전년동기)(%)'에 존재하는 문자 데이터: []
'총자본증가율(전년동기)(%)'에 존재하는 문자 데이터: ['당기잠식', '전기잠식', '잠식지속']
'유형자산증가율(전년동기)(%)'에 존재하는 문자 데이터: []
'영업이익증가율(전년동기)(%)'에 존재하는 문자 데이터: ['적전', '흑전', '적지']
'당기순이익증가율(전년동기)(%)'에 존재하는 문자 데이터: ['적전', '흑전', '적지']
'유형자산구성비율(%)'에 존재하는 문자 데이터: []
'무형자산구성비율(%)'에 존재하는 문자 데이터: []
'*판관비율(%)'에 존재하는 문자 데이터: []
'타인자본비율(%)'에 존재하는 문자 데이터: []
'부채비율(%)'에 존재하는 문자 데이터: ['완전잠식']
'현금및현금성자산구성비율(%)'에 존재하는 문자 데이터: []
'총자산회전율(회)'에 존재하는 문자 데이터: []
'총부채회전율(회)'에 존재하는 문자 데이터: []
'총자본회전율(회)'에 존재하는 문자 데이터: ['완전잠식']
'자본금회전율(회)'에 존재하는 문자 데이터: []
'현금흐름/총부채(%)'에 존재하는 문자 데이터: []
'현금흐름/총차입부채(%)'에 존재하는 문자 데이터: []
'현금흐름/영업수익(%)'에 존재하는 문자 데이터: []
'현금흐름/총자본(%)'에 존재하는 문자 데이터: ['완전잠식']
'유동비율(%)'에 존재하는 문자 데이터: []
'당좌비율(%)'에 존재하는 문자 데이터: []
'매출채권회전율'에

In [24]:
df.columns

Index(['Symbol', 'Name', '상장연도', '결산월', '회계년', '주기', '당기순이익률(%)', '매출총이익률(%)',
       '영업이익률(%)', '자본금영업이익률(%)', '자본금세전계속사업이익률(%)', '*영업수익/영업비용(%)',
       'ROE(세전계속사업이익)(%)', '자본금지배주주순이익률(%)', '매출액증가율(전년동기)(%)',
       '총자본증가율(전년동기)(%)', '유형자산증가율(전년동기)(%)', '영업이익증가율(전년동기)(%)',
       '당기순이익증가율(전년동기)(%)', '유형자산구성비율(%)', '무형자산구성비율(%)', '*판관비율(%)',
       '타인자본비율(%)', '부채비율(%)', '현금및현금성자산구성비율(%)', '총자산회전율(회)', '총부채회전율(회)',
       '총자본회전율(회)', '자본금회전율(회)', '현금흐름/총부채(%)', '현금흐름/총차입부채(%)',
       '현금흐름/영업수익(%)', '현금흐름/총자본(%)', '유동비율(%)', '당좌비율(%)', '매출채권회전율',
       '재고자산회전율', '고정자산회전율', '영업현금흐름/투자현금흐름(%)', 'DSRI', 'GMI', 'AQI', 'SGI',
       'DEPI', 'SGAI', 'LVGI', 'TATA', '벤포드', '분식기업'],
      dtype='object')

In [23]:
strtype_df = df[['Symbol', 'Name', '상장연도', '결산월', '회계년', '주기', 'ROE(세전계속사업이익)(%)', '총자본증가율(전년동기)(%)',
    '영업이익증가율(전년동기)(%)', '당기순이익증가율(전년동기)(%)', '부채비율(%)', '총자본회전율(회)', '현금흐름/총자본(%)']]

In [26]:
# strtype_df.to_csv('코스피_한글포함데이터확인.csv',index=False,encoding='UTF-8-sig')

In [25]:
value = pd.read_csv('../data/dataset/코스피_연결재무제표계정값.csv')
value.drop(columns = ['이자비용(천원)',
       '유동자산(천원)', '유동부채(천원)', '당좌자산(천원)', '매출액(천원)', '매출채권(천원)', '매출원가(천원)',
       '재고자산(천원)', '유형자산(천원)', '총자산(천원)', '판매비와관리비(천원)',
       '비유동부채(천원)', '영업활동으로인한현금흐름(천원)', '유무형자산상각비(천원)',
       '비유동자산(천원)', '투자활동으로인한현금흐름(천원)'], inplace = True)

check = pd.merge(df, value, on = ['Symbol', 'Name', '결산월', '회계년', '주기'], how = 'left')
check.drop(columns = ['당기순이익률(%)', '매출총이익률(%)',
       '영업이익률(%)', '자본금영업이익률(%)', '자본금세전계속사업이익률(%)', '*영업수익/영업비용(%)',
       '자본금지배주주순이익률(%)', 
       '유형자산증가율(전년동기)(%)',
       '유형자산구성비율(%)', '무형자산구성비율(%)', '*판관비율(%)',
       '타인자본비율(%)', '현금및현금성자산구성비율(%)', '총자산회전율(회)', '총부채회전율(회)',
       '자본금회전율(회)', '현금흐름/총부채(%)', '현금흐름/총차입부채(%)',
       '현금흐름/영업수익(%)', '유동비율(%)', '당좌비율(%)', '매출채권회전율',
       '재고자산회전율', '고정자산회전율', '영업현금흐름/투자현금흐름(%)', 'DSRI', 'GMI', 'AQI', 'SGI',
       'DEPI', 'SGAI', 'LVGI', 'TATA', '벤포드', '분식기업'], inplace = True)

In [31]:
check['영업이익(천원)_전기'] = check.groupby('Name')['영업이익(천원)'].shift(1)
check['당기순이익(천원)_전기'] = check.groupby('Name')['당기순이익(천원)'].shift(1)
check['자본총계_전기'] = check.groupby('Name')['총자본(천원)'].shift(1)

In [32]:
df['ROE(세전계속사업이익)(%)']

0           11.23
1            6.94
2            0.18
3          -10.52
4           25.51
          ...    
7308       -18.08
7309      -105.00
7310    -2,309.94
7311        51.53
7312        15.95
Name: ROE(세전계속사업이익)(%), Length: 7313, dtype: object

In [37]:
# 문자열 값이 있는 행을 식별 (예: 'ROE(세전계속사업이익)(%)' 열)
mask_roe = df['ROE(세전계속사업이익)(%)'].apply(lambda x: isinstance(x, str))
df[mask_roe]['ROE(세전계속사업이익)(%)']

3           11.23
4            6.94
5            0.18
6          -10.52
7           25.51
          ...    
8718       -18.08
8719      -105.00
8720    -2,309.94
8721        51.53
8722        15.95
Name: ROE(세전계속사업이익)(%), Length: 7268, dtype: object

In [27]:
# ROE(세전계속사업이익)(%): (세전계속사업이익 / 자본총계 * 100
check['ROE(세전계속사업이익)(%)_make'] = round((check['세전계속사업이익(천원)']/check['총자본(천원)'])*100, 2)

# 총자본증가율: 자본총계[당기]/자본총계[전기]-1
check['총자본증가율(전년동기)(%)_make'] = round(((check['총자본(천원)']-check['자본총계_전기'])/check['자본총계_전기'])*100, 2)


# 영업이익증가율: (영업이익[당기]-영업이익[전기])/영업이익[전기]
check['영업이익증가율(전년동기)(%)_make'] = round(((check['영업이익(천원)']-check['영업이익(천원)_전기'])/check['영업이익(천원)_전기'])*100, 2)

# 순이익증가율: (당기순이익[당기]-당기순이익[전기])/당기순이익[전기]
check['당기순이익증가율(전년동기)(%)_make'] = round(((check['당기순이익(천원)']-check['당기순이익(천원)_전기'])/check['당기순이익(천원)_전기'])*100, 2)


# 부채비율: 부채총계/자본총계
check['부채비율(%)_make'] = round((check['총부채(천원)']/check['총자본(천원)'])*100, 2)

In [28]:
check[['Symbol', 'Name', '회계년', 'ROE(세전계속사업이익)(%)', 'ROE(세전계속사업이익)(%)_make']]

,Symbol,Name,회계년,ROE(세전계속사업이익)(%),ROE(세전계속사업이익)(%)_make
0,A095570,AJ네트웍스,2015,11.23,9.61
1,A095570,AJ네트웍스,2016,6.94,6.70
2,A095570,AJ네트웍스,2017,0.18,0.18
3,A095570,AJ네트웍스,2018,-10.52,-10.05
4,A095570,AJ네트웍스,2019,25.51,30.82
...,...,...,...,...,...
7308,A003280,흥아해운,2018,-18.08,-25.85
7309,A003280,흥아해운,2019,-105.00,-176.71
7310,A003280,흥아해운,2020,"-2,309.94",203.70
7311,A003280,흥아해운,2021,51.53,19.19


In [46]:
value

,Symbol,Name,결산월,회계년,주기,영업이익(천원),총자본(천원),총부채(천원),당기순이익(천원),세전계속사업이익(천원),영업이익(천원)_전기,당기순이익(천원)_전기,자본총계_전기,ROE(세전계속사업이익)(%),총자본증가율(전년동기)(%),영업이익증가율(전년동기)(%),당기순이익증가율(전년동기)(%),부채비율(%)
0,A005930,삼성전자,12,2011,Annual,1.564429e+10,1.013136e+11,5.448663e+10,1.375904e+10,1.719192e+10,NaN,NaN,NaN,16.969008,NaN,NaN,NaN,53.780161
1,A005930,삼성전자,12,2012,Annual,2.904934e+10,1.214802e+11,5.959136e+10,2.384528e+10,2.991502e+10,1.564429e+10,1.375904e+10,1.013136e+11,24.625425,19.905097,85.686510,73.306276,49.054382
2,A005930,삼성전자,12,2013,Annual,3.678501e+10,1.500160e+11,6.405901e+10,3.047476e+10,3.836428e+10,2.904934e+10,2.384528e+10,1.214802e+11,25.573456,23.490085,26.629436,27.802054,42.701448
3,A005930,삼성전자,12,2014,Annual,2.502507e+10,1.680882e+11,6.233477e+10,2.339436e+10,2.787503e+10,3.678501e+10,3.047476e+10,1.500160e+11,16.583577,12.046833,-31.969384,-23.233670,37.084563
4,A005930,삼성전자,12,2015,Annual,2.641344e+10,1.790598e+11,6.311972e+10,1.906014e+10,2.596100e+10,2.502507e+10,2.339436e+10,1.680882e+11,14.498505,6.527298,5.547920,-18.526749,35.250634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13351,A950070,중국고섬,12,2018,Annual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13352,A950070,중국고섬,12,2019,Annual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13353,A950070,중국고섬,12,2020,Annual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13354,A950070,중국고섬,12,2021,Annual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
df.columns[:7]

Index(['Symbol', 'Name', '상장연도', '결산월', '회계년', '주기', '당기순이익률(%)'], dtype='object')

In [58]:
kospi = df.copy()
check = pd.DataFrame()
check['Symbol'] = kospi['Symbol']
check['Name'] = kospi['Name']
check['회계년'] = kospi['회계년']
check['ROE(세전계속사업이익)(%)'] = pd.to_numeric(kospi['ROE(세전계속사업이익)(%)'], errors='coerce')
check['매출액증가율(전년동기)(%)'] = pd.to_numeric(kospi['매출액증가율(전년동기)(%)'], errors='coerce')
check['총자본증가율(전년동기)(%)'] = pd.to_numeric(kospi['총자본증가율(전년동기)(%)'], errors='coerce')
check['영업이익증가율(전년동기)(%)'] = pd.to_numeric(kospi['영업이익증가율(전년동기)(%)'], errors='coerce')
check['당기순이익증가율(전년동기)(%)'] = pd.to_numeric(kospi['당기순이익증가율(전년동기)(%)'], errors='coerce')
check['부채비율(%)'] = pd.to_numeric(kospi['부채비율(%)'], errors='coerce')
check['총자본회전율(회)'] = pd.to_numeric(kospi['총자본회전율(회)'], errors='coerce')
check['현금흐름/총자본(%)'] = pd.to_numeric(kospi['현금흐름/총자본(%)'], errors='coerce')

In [60]:
check['ROE(세전계속사업이익)(%)']

3        11.23
4         6.94
5         0.18
6       -10.52
7        25.51
         ...  
8718    -18.08
8719   -105.00
8720       NaN
8721     51.53
8722     15.95
Name: ROE(세전계속사업이익)(%), Length: 7313, dtype: float64

In [40]:
df['ROE(세전계속사업이익)(%)'].value_counts()

ROE(세전계속사업이익)(%)
완전잠식      24
4.52       9
7.74       9
3.75       9
6.02       9
          ..
-66.58     1
5.31       1
12.01      1
21.87      1
15.95      1
Name: count, Length: 3727, dtype: int64

In [38]:
value

,Symbol,Name,결산월,회계년,주기,영업이익(천원),총자본(천원),총부채(천원),당기순이익(천원),세전계속사업이익(천원),영업이익(천원)_전기,당기순이익(천원)_전기,자본총계_전기,ROE(세전계속사업이익)(%),총자본증가율(전년동기)(%),영업이익증가율(전년동기)(%),당기순이익증가율(전년동기)(%),부채비율(%)
0,A005930,삼성전자,12,2011,Annual,1.564429e+10,1.013136e+11,5.448663e+10,1.375904e+10,1.719192e+10,NaN,NaN,NaN,16.969008,NaN,NaN,NaN,53.780161
1,A005930,삼성전자,12,2012,Annual,2.904934e+10,1.214802e+11,5.959136e+10,2.384528e+10,2.991502e+10,1.564429e+10,1.375904e+10,1.013136e+11,24.625425,19.905097,85.686510,73.306276,49.054382
2,A005930,삼성전자,12,2013,Annual,3.678501e+10,1.500160e+11,6.405901e+10,3.047476e+10,3.836428e+10,2.904934e+10,2.384528e+10,1.214802e+11,25.573456,23.490085,26.629436,27.802054,42.701448
3,A005930,삼성전자,12,2014,Annual,2.502507e+10,1.680882e+11,6.233477e+10,2.339436e+10,2.787503e+10,3.678501e+10,3.047476e+10,1.500160e+11,16.583577,12.046833,-31.969384,-23.233670,37.084563
4,A005930,삼성전자,12,2015,Annual,2.641344e+10,1.790598e+11,6.311972e+10,1.906014e+10,2.596100e+10,2.502507e+10,2.339436e+10,1.680882e+11,14.498505,6.527298,5.547920,-18.526749,35.250634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13351,A950070,중국고섬,12,2018,Annual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13352,A950070,중국고섬,12,2019,Annual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13353,A950070,중국고섬,12,2020,Annual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13354,A950070,중국고섬,12,2021,Annual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
# ROE(세전계속사업이익)(%): (세전계속사업이익 / 자본총계 * 100
df['ROE(세전계속사업이익)(%)'] = df['ROE(세전계속사업이익)(%)'].mask(df['ROE(세전계속사업이익)(%)'] == '완전잠식',
                                                     (value['세전계속사업이익(천원)']/value['총자본(천원)'])*100)

# 총자본증가율: 자본총계[당기]/자본총계[전기]-1
df['총자본증가율(전년동기)(%)'] = df['총자본증가율(전년동기)(%)'].mask(df['총자본증가율(전년동기)(%)'].isin
                                                   (['당기잠식', '전기잠식', '잠식지속']),
                                                   ((value['총자본(천원)']-value['자본총계_전기'])/value['자본총계_전기'])*100)


# 영업이익증가율: (영업이익[당기]-영업이익[전기])/영업이익[전기]
df['영업이익증가율(전년동기)(%)'] = df['영업이익증가율(전년동기)(%)'].mask(df['영업이익증가율(전년동기)(%)'].isin
                                                     (['적전', '흑전', '적지']),
                                       ((value['영업이익(천원)']-value['영업이익(천원)_전기'])/value['영업이익(천원)_전기'])*100)

# 순이익증가율: (당기순이익[당기]-당기순이익[전기])/당기순이익[전기]
df['당기순이익증가율(전년동기)(%)'] = df['당기순이익증가율(전년동기)(%)'].mask(df['당기순이익증가율(전년동기)(%)'].isin
                                                       (['적전', '흑전', '적지']),
                                       ((value['당기순이익(천원)']-value['당기순이익(천원)_전기'])/value['당기순이익(천원)_전기'])*100)


# 부채비율: 부채총계/자본총계
# 부채비율 계산 (0으로 나누는 경우를 피하기 위해 np.where 사용)
# df['계산된 부채비율'] = (df['부채총계'] / df['자본총계'].replace(0, pd.NA)) * 100
df['부채비율(%)'] = df['부채비율(%)'].mask(df['부채비율(%)'] == '완전잠식', (value['총부채(천원)']/value['총자본(천원)'])*100)

# 자본 음수값으로 최소값 0으로 대체
df['총자본회전율(회)'] = df['총자본회전율(회)'].mask(df['총자본회전율(회)'] == '완전잠식', 0)
df['현금흐름/총자본(%)'] = df['현금흐름/총자본(%)'].mask(df['현금흐름/총자본(%)'] == '완전잠식', 0)

In [98]:
import re

columns_to_check = df.columns[6:-2]

# 각 열에 대해 고유한 값 추출하여 한글을 포함하는 값 확인
for column in columns_to_check:
    # 고유한 값 추출
    unique_values = df[column].astype(str).unique()
    # 한글을 포함하는 값 추출
    korean_values = [value for value in unique_values if re.search('[ㄱ-ㅎㅏ-ㅣ가-힣]', value)]
    print(f"'{column}'에 존재하는 문자 데이터: {korean_values}")

'당기순이익률(%)'에 존재하는 문자 데이터: []
'매출총이익률(%)'에 존재하는 문자 데이터: []
'영업이익률(%)'에 존재하는 문자 데이터: []
'자본금영업이익률(%)'에 존재하는 문자 데이터: []
'자본금세전계속사업이익률(%)'에 존재하는 문자 데이터: []
'*영업수익/영업비용(%)'에 존재하는 문자 데이터: []
'ROE(세전계속사업이익)(%)'에 존재하는 문자 데이터: []
'자본금지배주주순이익률(%)'에 존재하는 문자 데이터: []
'매출액증가율(전년동기)(%)'에 존재하는 문자 데이터: []
'총자본증가율(전년동기)(%)'에 존재하는 문자 데이터: []
'유형자산증가율(전년동기)(%)'에 존재하는 문자 데이터: []
'영업이익증가율(전년동기)(%)'에 존재하는 문자 데이터: []
'당기순이익증가율(전년동기)(%)'에 존재하는 문자 데이터: []
'유형자산구성비율(%)'에 존재하는 문자 데이터: []
'무형자산구성비율(%)'에 존재하는 문자 데이터: []
'*판관비율(%)'에 존재하는 문자 데이터: []
'타인자본비율(%)'에 존재하는 문자 데이터: []
'부채비율(%)'에 존재하는 문자 데이터: []
'현금및현금성자산구성비율(%)'에 존재하는 문자 데이터: []
'총자산회전율(회)'에 존재하는 문자 데이터: []
'총부채회전율(회)'에 존재하는 문자 데이터: []
'총자본회전율(회)'에 존재하는 문자 데이터: []
'자본금회전율(회)'에 존재하는 문자 데이터: []
'현금흐름/총부채(%)'에 존재하는 문자 데이터: []
'현금흐름/영업수익(%)'에 존재하는 문자 데이터: []
'현금흐름/총자본(%)'에 존재하는 문자 데이터: []
'유동비율(%)'에 존재하는 문자 데이터: []
'당좌비율(%)'에 존재하는 문자 데이터: []
'매출채권회전율'에 존재하는 문자 데이터: []
'재고자산회전율'에 존재하는 문자 데이터: []
'고정자산회전율'에 존재하는 문자 데이터: []
'영업현금흐름/투자현금흐름(%)'에 존재하는 문자 데이터: []
'DSR

---
### 결측치 / 무한대 값 처리
모든 결측치가 label = 0인 기업이기 때문에
- 결측치(N/A(IFRS) 포함) 제거
- 무한대 값 (inf/-inf) -> 결측치 -> 제거

In [99]:
df[df['자본금영업이익률(%)'].isnull()]['분식기업'].value_counts()

분식기업
0.0    45
Name: count, dtype: int64

In [113]:
for col in df.columns[6:-2]:
    df[col] = df[col].replace(',', '', regex=True)  # 콤마 제거
    df[col] = pd.to_numeric(df[col], errors='coerce')

C:\Users\Master\AppData\Local\Temp\ipykernel_18632\621108049.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(',', '', regex=True)  # 콤마 제거


In [126]:
df.isin([np.inf, -np.inf]).sum()

Symbol                 0
Name                   0
상장연도                   0
결산월                    0
회계년                    0
주기                     0
당기순이익률(%)              0
매출총이익률(%)              0
영업이익률(%)               0
자본금영업이익률(%)            0
자본금세전계속사업이익률(%)        0
*영업수익/영업비용(%)          0
ROE(세전계속사업이익)(%)       0
자본금지배주주순이익률(%)         0
매출액증가율(전년동기)(%)        0
총자본증가율(전년동기)(%)        0
유형자산증가율(전년동기)(%)       0
영업이익증가율(전년동기)(%)       1
당기순이익증가율(전년동기)(%)      1
유형자산구성비율(%)            0
무형자산구성비율(%)            0
*판관비율(%)               0
타인자본비율(%)              0
부채비율(%)                0
현금및현금성자산구성비율(%)        0
총자산회전율(회)              0
총부채회전율(회)              0
총자본회전율(회)              0
자본금회전율(회)              0
현금흐름/총부채(%)            0
현금흐름/영업수익(%)           0
현금흐름/총자본(%)            0
유동비율(%)                0
당좌비율(%)                0
매출채권회전율               23
재고자산회전율              130
고정자산회전율                0
영업현금흐름/투자현금흐름(%)       0
DSRI                  18
GMI                    0


In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7313 entries, 3 to 8722
Data columns (total 48 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Symbol             7313 non-null   object 
 1   Name               7313 non-null   object 
 2   상장연도               7313 non-null   int32  
 3   결산월                7313 non-null   int64  
 4   회계년                7313 non-null   int64  
 5   주기                 7313 non-null   object 
 6   당기순이익률(%)          7312 non-null   float64
 7   매출총이익률(%)          7312 non-null   float64
 8   영업이익률(%)           7312 non-null   float64
 9   자본금영업이익률(%)        7268 non-null   float64
 10  자본금세전계속사업이익률(%)    7268 non-null   float64
 11  *영업수익/영업비용(%)      7313 non-null   float64
 12  ROE(세전계속사업이익)(%)   7265 non-null   float64
 13  자본금지배주주순이익률(%)     7268 non-null   float64
 14  매출액증가율(전년동기)(%)    7266 non-null   float64
 15  총자본증가율(전년동기)(%)    7257 non-null   float64
 16  유형자산증가율(전년동기)(%)   7268 non-n

In [118]:
import numpy as np

# 6번째부터 -2번째까지의 열에서 무한대 값 확인
inf_mask = np.isinf(df.iloc[:, 6:-2])

# 무한대 값이 있는 행 출력
inf_rows = df[inf_mask.any(axis=1)]

print("Rows with infinite values:")
print(inf_rows)

Rows with infinite values:
       Symbol      Name  상장연도  결산월   회계년      주기  당기순이익률(%)  매출총이익률(%)  \
202   A069730     DSR제강  2003   12  2018  Annual       3.99       9.56   
339   A035000      HS애드  1999   12  2012  Annual       3.66      31.39   
340   A035000      HS애드  1999   12  2013  Annual       3.56      32.69   
341   A035000      HS애드  1999   12  2014  Annual       2.47      31.01   
342   A035000      HS애드  1999   12  2015  Annual       2.42      28.13   
...       ...       ...   ...  ...   ...     ...        ...        ...   
7814  A009180   한솔로지스틱스  1989   12  2019  Annual       1.28       8.20   
7815  A009180   한솔로지스틱스  1989   12  2020  Annual       1.70       8.31   
7816  A009180   한솔로지스틱스  1989   12  2021  Annual       3.14       8.59   
8155  A042660      한화오션  2001   12  2012  Annual      -2.06       4.83   
8486  A241590  화승엔터프라이즈  2016   12  2016  Annual       5.46      14.60   

      영업이익률(%)  자본금영업이익률(%)  ...      DSRI       GMI       AQI       SGI  \
202     

In [122]:
inf_rows_df = df[inf_mask.any(axis=1)].copy()

In [123]:
inf_rows_df

,Symbol,Name,상장연도,결산월,회계년,주기,당기순이익률(%),매출총이익률(%),영업이익률(%),자본금영업이익률(%),...,DSRI,GMI,AQI,SGI,DEPI,SGAI,LVGI,TATA,벤포드,분식기업
202,A069730,DSR제강,2003,12,2018,Annual,3.99,9.56,3.37,86.63,...,inf,1.032937,1.104331,1.043419,0.001312,0.730324,1.001583,-0.019729,0,0.0
339,A035000,HS애드,1999,12,2012,Annual,3.66,31.39,4.50,85.98,...,1.023131,0.953486,1.003392,1.036307,0.364513,1.044471,0.966467,0.084270,0,1.0
340,A035000,HS애드,1999,12,2013,Annual,3.56,32.69,4.62,91.82,...,1.080980,0.960245,0.856235,1.039417,0.358633,1.043740,1.019264,0.022405,0,1.0
341,A035000,HS애드,1999,12,2014,Annual,2.47,31.01,3.33,73.07,...,0.868613,1.053989,1.148991,1.104600,0.413144,0.986378,0.975948,0.015507,0,1.0
342,A035000,HS애드,1999,12,2015,Annual,2.42,28.13,2.88,78.58,...,1.301790,1.102469,0.894675,1.242981,0.425077,0.912089,1.051421,-0.008447,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7814,A009180,한솔로지스틱스,1989,12,2019,Annual,1.28,8.20,2.07,66.38,...,0.947346,0.793629,1.796755,1.061195,0.075657,1.093610,1.290974,-0.118658,1,0.0
7815,A009180,한솔로지스틱스,1989,12,2020,Annual,1.70,8.31,2.57,91.18,...,1.120526,0.986637,0.947331,1.106099,0.223538,0.936615,1.004619,-0.129839,0,0.0
7816,A009180,한솔로지스틱스,1989,12,2021,Annual,3.14,8.59,4.06,211.32,...,0.974795,0.968012,0.804908,1.465904,0.193335,0.788318,0.977973,0.064218,0,0.0
8155,A042660,한화오션,2001,12,2012,Annual,-2.06,4.83,-0.53,-7.49,...,inf,2.333937,1.119938,0.976929,0.000000,1.250023,1.012101,0.059570,0,1.0


In [125]:
inf_rows_df.to_csv('무한대.csv', encoding='UTF-8-sig', index = False)